- 이 notebook은 LR-GCCF(Linear Residual Graph Convolutional Collaborative Filtering)라는 model을 다룹니다[(AAAI2020)](https://arxiv.org/abs/2001.10167). [Github](https://github.com/newlei/LR-GCCF)에 저자가 구현해둔 code가 있지만 sub-classing으로 model을 선언할 때 다른 model 이름을 적어놓을 정도로 정리가 안 되어 있는 부분도 있고, optimizer가 아니라 model에 대해 ```zero_grad()```를 해놓는 등 잘못된 부분도 있어 논문을 읽고 code에 주석을 달아가며 정리하였습니다.
- 사실 graph data에 적용할 수 있는 많은 model들이 이미 torch_geometric이라는 Library에 이미 구현이 되어 있고 이를 colab으로 실습도 해봤습니다. 하지만 전체 과정을 밑바닥부터 작성해보면서 공부해보기 위해, 일부러 해당 library의 도움을 받지 않고 구현해보았습니다.
- LR-GCCF를 간단히 설명하자면 user와 item으로 양분된 Bipartite graph를 input으로 받아, 각 user와 item을 지정된 dimension의 vector로 embedding하는 model입니다. LR-GCCF는 GCN(Graph Convolutional Network)의 변형된 model이며, GCN에서 non-linear activation function을 제거한 경량 model인 Simplifying GCN에(L; Linear) 추가적인 구조를(R; Residual) 더해 성능을 소폭 향상시킨 model입니다. 아래에 model이 정의된 부분에서 더 자세히 설명하겠지만 해당 model을 학습시키기 위해 BPR(Bayesian Personalized Ranking) loss를 사용합니다.
- model 이름이 Collaborative Filtering인 만큼 해당 model은 recommender system으로 활용하도록 고안되었습니다. 사용방법은 간단한데요, model의 output으로 나오는 embedding된 user와 item vector들을 inner product하여, 결과 값이 가장 큰 item들을 순서대로 추천하는 방법을 사용합니다.

In [ ]:
# -- coding:UTF-8
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader

import os
import numpy as np
import time

# Not standard library
import data_utils 
import evaluate

In [ ]:
# CUDA Setting
GPU_NUM = 7 # No special meaning. It's my GPU assigned from Lab...
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

print('Device:', device)
print('Count of using GPUs:', torch.cuda.device_count())
print('Current cuda device:', torch.cuda.current_device())

In [ ]:
# Train result path setting
result_base_path = '../result'
model_name = 'LRGCCF_amazon_books_2018'
training_result_path = result_base_path + '/' + model_name + '/training_result'
if(os.path.exists(training_result_path)):
    print("Already result paths exist")
else:
    print("Make model result path : ", model_name)
    os.makedirs(training_result_path)
training_result_file = open(training_result_path + '/results.txt', 'w+')

In [ ]:
# Load data
data_base_path = '../data'
training_user_set, training_item_set, training_set_count = np.load(data_base_path + '/training_set.npy', allow_pickle=True)
testing_user_set, testing_item_set, testing_set_count = np.load(data_base_path + '/testing_set.npy', allow_pickle=True)  
user_rating_set_all = np.load(data_base_path + '/rating_set_all.npy', allow_pickle=True).item()

In [ ]:
# Arguments
args = {
    'lr': 0.001,
    'epochs': 351,
    'user_num' : 73058,
    'item_num' : 160808,
    'factor_num' : 64,
    'batch_size' : 2048*300
}

- 사용하려는 data는 이전에 Amazon-Books를 전처리한 data로, 총 73,058명의 user와 160,808개의 item으로 구성되어있습니다. ```factor_num```은 해당 model의 output으로 return될 embedding vector의 dimension을 의미합니다.

In [ ]:
# Data processing 
### Load Degree and return reciprocal of degree.
print("Make reciprocal of degree vector")
def readD(set_matrix,num_): 
    user_d = [] 
    for i in range(num_):
        len_set = 1.0 / (len(set_matrix[i])+1)
        user_d.append(len_set)
    return user_d

u_d = readD(training_user_set, args['user_num'])
i_d = readD(training_item_set, args['item_num'])

- model을 training하는 데에 필요한 것으로, 각 user(or item)가 몇개의 item(or user)과 연결되어 있는지를 degree로 간주하여 이에 대한 역수를 저장하는 함수입니다. 왜 역수를 저장하냐면, user를 embedding할 때 연결된 item들을 aggregating하여 나타내는데, item embedding들을 mean으로 aggregate하기 위해 degree의 역수를 저장하는 것입니다.

In [ ]:
### Make adjacency matrix with reciprocal of degree; 1/(sqrt(degree(i)*degree(j)), i for user j for item
print("Make sparse adjacency matrix")
def readTrainSparseMatrix(set_matrix, is_user):
    user_items_matrix_i = []
    user_items_matrix_v = [] 
    if is_user:
        d_i = u_d
        d_j = i_d
    else:
        d_i = i_d
        d_j = u_d
    for i in set_matrix:
        for j in set_matrix[i]: 
            user_items_matrix_i.append([i,j])
            d_i_j = np.sqrt(d_i[i]*d_j[j]) 
            user_items_matrix_v.append(d_i_j) 

    user_items_matrix_i = torch.cuda.LongTensor(user_items_matrix_i)
    user_items_matrix_v = torch.cuda.FloatTensor(user_items_matrix_v)
    return torch.sparse.FloatTensor(user_items_matrix_i.t(), user_items_matrix_v) 

sparse_u_i = readTrainSparseMatrix(training_user_set, True)
sparse_i_u = readTrainSparseMatrix(training_item_set, False) 

- 이 역시 model을 training하는 데에 필요한 것으로, 위에서 언급한 user의 degree의 역수와 item의 degree의 역수를 곱한 후 square root를 씌워준 값을 element$({1 \over \sqrt{d_u \times d_i}}; d_u: user\ degree,\ d_i: item\ degree)$로 갖는 matrix를 생성합니다. 각 matrix는 \[user $\times$ item\](or \[item $\times$ user\])의 shape을 가집니다.

In [ ]:
# LRGCCF class
class LRGCCF(nn.Module):
    def __init__(self, user_num, item_num, factor_num, user_item_matrix, item_user_matrix, user_deg_rec, item_deg_rec):
        super(LRGCCF, self).__init__()
        
        self.user_item_matrix = user_item_matrix
        self.item_user_matrix = item_user_matrix

        self.user_deg_rec = torch.cuda.FloatTensor(user_deg_rec)
        self.item_deg_rec = torch.cuda.FloatTensor(item_deg_rec)
        self.user_deg_rec = torch.unsqueeze(self.user_deg_rec, 1)
        self.item_deg_rec = torch.unsqueeze(self.item_deg_rec, 1)
        self.user_deg_rec = self.user_deg_rec.expand(-1, factor_num)
        self.item_deg_rec = self.item_deg_rec.expand(-1, factor_num)
        
        self.embed_user = nn.Embedding(user_num, factor_num)
        self.embed_item = nn.Embedding(item_num, factor_num) 
        nn.init.normal_(self.embed_user.weight, std=0.01)
        nn.init.normal_(self.embed_item.weight, std=0.01)

    def forward(self, user, item_i, item_j):    

        users_embedding = self.embed_user.weight
        items_embedding = self.embed_item.weight

        gcn1_users_embedding = (torch.sparse.mm(self.user_item_matrix, items_embedding) + users_embedding.mul(self.user_deg_rec))
        gcn1_items_embedding = (torch.sparse.mm(self.item_user_matrix, users_embedding) + items_embedding.mul(self.item_deg_rec))

        gcn2_users_embedding = (torch.sparse.mm(self.user_item_matrix, gcn1_items_embedding) + gcn1_users_embedding.mul(self.user_deg_rec))
        gcn2_items_embedding = (torch.sparse.mm(self.item_user_matrix, gcn1_users_embedding) + gcn1_items_embedding.mul(self.item_deg_rec))

        gcn3_users_embedding = (torch.sparse.mm(self.user_item_matrix, gcn2_items_embedding) + gcn2_users_embedding.mul(self.user_deg_rec))
        gcn3_items_embedding = (torch.sparse.mm(self.item_user_matrix, gcn2_users_embedding) + gcn2_items_embedding.mul(self.item_deg_rec))

        gcn4_users_embedding = (torch.sparse.mm(self.user_item_matrix, gcn3_items_embedding) + gcn3_users_embedding.mul(self.user_deg_rec))
        gcn4_items_embedding = (torch.sparse.mm(self.item_user_matrix, gcn3_users_embedding) + gcn3_items_embedding.mul(self.item_deg_rec))
        
        gcn_total_users_embedding= torch.cat((users_embedding, gcn1_users_embedding, gcn2_users_embedding, gcn3_users_embedding, gcn4_users_embedding), -1)
        gcn_total_items_embedding= torch.cat((items_embedding, gcn1_items_embedding, gcn2_items_embedding, gcn3_items_embedding, gcn4_items_embedding), -1)

        # for loss calculation(Bayesian Personalized Ranking loss)
        user = F.embedding(user, gcn_total_users_embedding)
        item_i = F.embedding(item_i, gcn_total_items_embedding) 
        item_j = F.embedding(item_j, gcn_total_items_embedding) 

        prediction_i = (user * item_i).sum(dim=-1)
        prediction_j = (user * item_j).sum(dim=-1) 
        l2_regulization = 0.01*(user**2 + item_i**2 + item_j**2).sum(dim=-1)
        
        loss_no_reg = -((prediction_i - prediction_j)).sigmoid().log().mean()
        loss = loss_no_reg + l2_regulization.mean()

        return prediction_i, prediction_j, loss, loss_no_reg

- 결국 학습시키는 parameter는 embedding layer입니다. 별도의 추가적인 parameter나 non-linearity를 더하는 activation function이 없어 빠르게 training할 수 있다는 것이 해당 model의 가장 큰 장점입니다.

- forward 설명
- degree들의 역수의 곱을 element로 가지는 ```user_item_matrix```(\[user $\times$ item\])와 ```items_embedding```(\[item $\times$ n\])을 matrix multiplication을 하면, \[user $\times$ n\]의 shape을 가진 embedding matrix가 나옵니다. 해당 matrix의 element가 가지는 의미는, 특정 user가 특정 item들과 어떻게 연결되었는지를 나타내는 ```user_item_matrix```을 참조해서, item이 embedding된 ```items_embedding```를 aggregate하여 user의 embedding vector를 표현합니다.
- 이렇게 생성된 user embedding에, 기존 user embedding에 user의 degree의 역수를 곱한 값을 더하여, self-reference까지 합니다.
- 상기의 두가지 계산을 item에도 똑같이 적용하여 item embedding도 같이 진행해줍니다.
- 이런 과정을 4번씩 반복하며 embedding layer를 만들어주는데, loss를 계산할 땐 4개의 layer를 concat하여 모두 사용합니다. 이렇게 하는 이유는 이전 layer의 embedding을 직접적으로 사용하여 skip-connection을 하는 효과를 얻을 수 있기 때문입니다. 저자는 이미지 인식에 유용한 model인 resNet에서 해당 아이디어를 착안했다고 합니다. 실험결과 해당 방법을 사용하는 것은 유의미한 성능향상을 이끌어내기에 유효한 방법인 것 같습니다.(test 시 약 5~10%의 metric값(HitRatio, nDCG) 개선)

- loss 설명
- loss는 맨 처음 설명했던대로  BPR(Bayesian Personalized Ranking) loss를 사용하는데, 이는 user에게 추천해야할 item인 ```item_i```와 추천하지 말아야 할 item인 ```item_j```의 inner product의 값 차이를 극대화하는 방법으로 진행됩니다. ```item_j```는 user가 rating을 하지 않았던 item들 중 무작위로 sampling하여 선정됩니다.
- 이에 더해 BPR loss엔 user와 item의 embedding 값에 제한을 두는 reguralization도 적용합니다.

In [ ]:
# Data Loader
train_dataset = data_utils.BPRData(
        train_dict=training_user_set, num_item=args['item_num'], num_ng=5, is_training=True,\
        data_set_count=training_set_count,all_rating=user_rating_set_all)
train_loader = DataLoader(train_dataset,
        batch_size=args['batch_size'], shuffle=True, num_workers=2)
  
testing_dataset_loss = data_utils.BPRData(
        train_dict=testing_user_set, num_item=args['item_num'], num_ng=5, is_training=True,\
        data_set_count=testing_set_count,all_rating=user_rating_set_all)
testing_loader_loss = DataLoader(testing_dataset_loss,
        batch_size=args['batch_size'], shuffle=False, num_workers=0)

In [ ]:
# Model 
model = LRGCCF(args['user_num'], args['item_num'], args['factor_num'], sparse_u_i, sparse_i_u, u_d, i_d)
model = model.to(device) 

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

In [ ]:
# Training
print('--------training processing-------')
for epoch in range(args['epochs']):
    model.train() 
    
    start_time = time.time()
    
    print('Negative sampling... ', end='')
    train_loader.dataset.ng_sample()
    print('end')

    print('--training--')
    train_loss_sum = []
    for user, item_i, item_j in train_loader:
        user = user.cuda()
        item_i = item_i.cuda()
        item_j = item_j.cuda() 

        optimizer.zero_grad()
        prediction_i, prediction_j, loss, loss_no_reg = model(user, item_i, item_j) 
        loss.backward()
        optimizer.step()

        train_loss_sum.append(loss.item())  

    elapsed_time = time.time() - start_time
    
    train_loss = round(np.mean(train_loss_sum[:-1]), 4)
    print_train_result = "epoch: " + str(epoch) + '\ttime: ' + str(round(elapsed_time,1)) + 's' + '\t train loss: ' + str(train_loss)
    
    # model save
    PATH_model = training_result_path + '/epoch' + str(epoch) + '.pt'
    torch.save(model.state_dict(), PATH_model)
    
    #######test and val###########
    model.eval()   
    testing_loader_loss.dataset.ng_sample() 
    print_test_loss = '\ttest loss: '+ str(evaluate.metrics_loss(model, testing_loader_loss, args['batch_size']))
    print(print_train_result + print_test_loss)
    
    #######test result save###########
    training_result_file.write(print_train_result + print_test_loss) 
    training_result_file.write('\n') 
    training_result_file.flush() 

- training과정은 단순합니다. 각 user가 rating 하지 않았던 item들인 ```item_j```를 sampling하고, training data의 user와 item_i, item_j를 model에 넣어 계산해주고, loss를 계산하여 optimizer로 embedding을 최적화합니다. training이 끝나면 train loss를 산출하고 model을 epoch에 따라 저장한 후, test data로 test loss를 계산하여 출력함으로써 마무리됩니다.

- 향후에 개선해볼 수 있는 사항으로는, self-reference를 하는 matrix를 바꿔볼 수 있을 것 같습니다. degree의 역수뿐만 아니라 numerical하게 matrix의 element로 사용할 수 있는 추가적인 feature가 있다면, heterogeneous한 graph data를 만들어 training을 진행해볼 수 있을 것 같습니다. 실제 서비스에 적용할 계획이라면, user의 성별, 접속 위치 등의 개인정보를 더해 표현할 수 있을 것 같습니다.
- 다른 개선사항으로는 ```item_j```를 sampling하는 방법이 있을 것 같습니다. 현재는 실제 user가 rating하지 않을 item들 중 무작위로 뽑아 생성하였지만, user가 rating하진 않았지만 rating한 item과 유사한 item들을 주로 뽑아 사용한다면 좀 더 sharp한 model이 될 수 있을 것 같습니다. 해당 아이디어는 Pinterest에서 graph를 이용하여 recommendation system을 구축한 방법을 담은 [Pinsage](https://arxiv.org/pdf/1806.01973.pdf)에서 착안했습니다.